In [10]:
import numpy as np
import pandas as pd

define activation functions

In [11]:
logistic=lambda x:1/(1+np.exp(-x))

d_logistic= lambda result: result*(1-result)

### Network classes

In [1]:
class HiddenLayer(object):
    def __init__(self,n_in, n_out, weights=None,bias=None):
        if weights=None:
            weights=2*np.random.random((n_in,n_out))-1
        if bias=None:
            bias=np.zeros((n_out),dtype=np.float64)
        self.weights=weights
        self.bias=bias
        self.output=np.zeros((n_out),dtype=np.float64)
        self.inputs=np.zeros((n_in),dtype=np.float64)
    def forward_prop(inputs):
        ins=self.bias+np.dot(self.weights,inputs)
        self.inputs=
        self.output = logistic(ins)
        return self.output
    def backprop(error):
        delta=np.dot(error*d_logistic(self.output))
        self.weights=
        
        

SyntaxError: invalid syntax (<ipython-input-1-42784cdbeb1f>, line 3)

In [23]:
class HiddenLayer(object):
    def __init__(self,n_in, n_out, weights=None):
        if weights==None:
            weights=2*np.random.random((n_in,n_out))-1
        #if bias=None:
        #    bias=np.zeros((n_out),dtype=np.float64)
        self.n_in=n_in
        self.n_out=n_out
        self.weights=weights
        #self.bias=bias
        self.output=np.zeros((n_out),dtype=np.float64)
        self.inputs=np.zeros((n_in),dtype=np.float64)
        self.delta=np.zeros((n_in),dtype=np.float64)
        self.error=np.zeros((n_out),dtype=np.float64)
    def forward_prop(self,inputs):
        ins=self.bias+np.dot(inputs,self.weights)
        #ins=np.dot(inputs,self.weights)
        self.inputs=inputs
        self.output = logistic(ins)
        #return self.output
    def backprop(self,delta,alpha):
        #delta=np.dot(error*d_logistic(self.output))
        self.error=delta.dot(self.weights.T)
        self.back_delta=self.error*d_logistic(self.inputs)
        self.weights+=alpha*self.inputs.T.dot(delta)
        #return self.back_delta
        
        

Add a bias

In [191]:
class HiddenLayer(object):
    def __init__(self,n_in, n_out, weights=None,bias=None):
        if weights==None:
            weights=(2*np.random.random((n_in,n_out))-1)
        if bias==None:
            bias=(np.zeros((n_out),dtype=np.float64))
        self.n_in=n_in
        self.n_out=n_out
        self.weights=weights
        self.bias=bias
        self.output=np.zeros((n_out),dtype=np.float64)
        self.inputs=np.zeros((n_in),dtype=np.float64)
        self.delta=np.zeros((n_in),dtype=np.float64)
        self.error=np.zeros((n_out),dtype=np.float64)
    def forward_prop(self,inputs):
        #ins=self.bias+np.dot(inputs,self.weights)
        ins=np.dot(inputs,self.weights)
        self.output = logistic(ins)
        self.inputs=inputs
        #return self.output
    def backprop(self,delta,alpha):
        #self.error=np.array(delta.dot(np.matrix(self.bias).T+self.weights.T))
        self.error=np.array(delta.dot(self.weights.T))
        self.back_delta=self.error*d_logistic(self.inputs)
        self.weights-=alpha*self.inputs.T.dot(delta)
        #self.bias-=alpha*delta.sum(axis=0)
        #return self.back_delta
        
        

In [192]:
class OutputLayer(object):
    def __init__(self,y):
        if (len(y.shape)>1):
            n_in=y.shape[-1]
        else:
            #needed in case you want to input a 
            y=np.expand_dims(y,axis=1)
            n_in=1
        self.n_in=n_in
        self.y=y
        self.inputs=np.zeros((n_in),dtype=np.float64)
    def forward_prop(self,inputs):
        self.inputs=inputs
    def backprop(self,alpha):
        self.error=self.y-self.inputs
        self.back_delta=self.error*d_logistic(self.inputs)
        

In [193]:
class Multi_Layer_Perceptron(object):
    def __init__(self,n_layers,features,y,alpha=0.1, n_hidden_paths=None):
        self.alpha=alpha
        self.features=features
        self.n_layers=n_layers
        #Set the number of features
        if (len(features.shape)>1):
            self.n_features=features.shape[-1]
        else:
            self.n_features=1
        #Set the number of output features
        if (len(y.shape)>1):
            n_out=y.shape[-1]
        else:
            n_out=1
        if n_hidden_paths==None:
            n_hidden_paths=self.n_features+1
        #Initialize the layers of the network
        if n_layers>2:
            self.layers=[HiddenLayer(self.n_features,n_hidden_paths)]
            if n_layers>3:
                self.layers.extend([HiddenLayer(n_hidden_paths,n_hidden_paths) for i in xrange(n_layers-3)])
            self.layers.extend([HiddenLayer(n_hidden_paths,n_out),OutputLayer(y)])
        elif n_layers==2:
            self.layers=[HiddenLayer(self.n_features,n_out),OutputLayer(y)]
        else:
            raise ValueError('Network must have at least 2 layers, an input and output layer.')
    def forward_prop(self):
        self.layers[0].forward_prop(self.features)
        for i in xrange(1,self.n_layers):
            self.layers[i].forward_prop(self.layers[i-1].output)
    def backprop(self,alpha=None):
        if alpha != None:
            self.alpha=alpha
        self.layers[-1].backprop(self.alpha)
        for i in reversed(xrange(0,self.n_layers-1)):
            self.layers[i].backprop(self.layers[i+1].back_delta,self.alpha)
            
        

### Some test data

In [172]:
with open('../datapath.txt') as f:
    datapath=f.readlines()[0].rstrip()

In [175]:
train=pd.read_csv(datapath+'/train.csv', nrows=1000, dtype=np.uint8)
pixels=train.columns.drop('label')

In [176]:
train=train[(train['label']==1)|(train['label']==0)]

In [202]:
MLP=Multi_Layer_Perceptron(20,train[pixels],train['label'])

In [203]:
%%timeit
MLP.forward_prop()
MLP.backprop()

1 loops, best of 3: 1.54 s per loop


In [ ]:
for i in xrange(100):
    MLP.forward_prop()
    MLP.backprop()

In [198]:
MLP.layers[-1].inputs.mean()

1.7436827433690933e-16

In [199]:
np.abs(MLP.layers[-1].error).mean()

0.47290640394088684

In [178]:
MLP.forward_prop()


In [179]:
MLP.backprop()

In [129]:
MLP.layers[1].bias.shape

(4,)

In [125]:
MLP.layers[0].weights.shape

(3, 4)

In [126]:
MLP.layers[0].backprop(MLP.layers[1].back_delta,0.1)

ValueError: operands could not be broadcast together with shapes (4,) (4,3) 

In [276]:
MLP.forward_prop()

In [277]:
MLP.backprop()

In [281]:
[x.error.sum() for x in MLP.layers]

[0.0, 0.0, 0.0, 0.0, -9.0]

In [164]:
MLP.layers[-2].backprop(MLP.layers[-1].back_delta)

In [169]:
pd.DataFrame(MLP.layers[-2].back_delta).isnull().sum().sum()

0

In [170]:
MLP.layers[-3].backprop(MLP.layers[-2].back_delta)

In [ ]:
MLP.layers[-3].

In [155]:
pd.DataFrame(MLP.layers[2].weights)[758].isnull()

0      True
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10     True
11     True
12     True
13     True
14     True
15     True
16     True
17     True
18     True
19     True
20     True
21     True
22     True
23     True
24     True
25     True
26     True
27     True
28     True
29     True
       ... 
755    True
756    True
757    True
758    True
759    True
760    True
761    True
762    True
763    True
764    True
765    True
766    True
767    True
768    True
769    True
770    True
771    True
772    True
773    True
774    True
775    True
776    True
777    True
778    True
779    True
780    True
781    True
782    True
783    True
784    True
Name: 758, dtype: bool

In [145]:
(pd.DataFrame(MLP.layers[3].back_delta)).isnull().sum().sum()

0

In [119]:
(MLP.layers[2].weights.T).shape

(785, 785)

In [128]:
MLP.layers[3].back_delta.shape

(25, 785)

In [46]:
X=np.array([[0,0,1],[0,1,1],[1,0,1],[1,1,1],[1,1,1]])
X.shape

(5, 3)

In [47]:
syn0=np.random.random((3,8))

In [48]:
np.dot(X,syn0)

array([[ 0.60444389,  0.70396007,  0.71766409,  0.56045141,  0.70625022,
         0.37125583,  0.40015699,  0.42586033],
       [ 0.91846872,  1.3745933 ,  0.87059902,  1.53323425,  0.85016604,
         1.3363059 ,  1.08538557,  0.99220826],
       [ 1.13431584,  0.86828019,  1.47118578,  1.42484416,  0.79117603,
         1.23680575,  0.45710503,  0.77771831],
       [ 1.44834067,  1.53891342,  1.62412071,  2.39762701,  0.93509185,
         2.20185582,  1.1423336 ,  1.34406624],
       [ 1.44834067,  1.53891342,  1.62412071,  2.39762701,  0.93509185,
         2.20185582,  1.1423336 ,  1.34406624]])

In [77]:
np.expand_dims(train['label'],axis=1)

array([[1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0]], dtype=uint8)

### Some simpler data

In [83]:
np.ones((1,5))

array([[ 1.,  1.,  1.,  1.,  1.]])

In [51]:
X=np.array([[0,0,1],[0,1,1],[1,0,1],[1,1,1],[1,1,1]])

In [52]:
y=np.array([[0],[1],[1],[0],[1]])

In [169]:
MLP=Multi_Layer_Perceptron(5,X,y)

In [170]:
MLP.forward_prop()

In [171]:
MLP.backprop()

In [162]:
np.matrix(MLP.layers[0].bias).T.shape

(4, 1)

In [70]:
MLP.layers[0].bias+np.dot(X,MLP.layers[0].weights)

array([[-0.41562947,  0.40602554, -0.25208644, -0.11352984],
       [-0.66647229,  0.95953573,  0.18824454,  0.67506626],
       [-0.65410452, -0.16710501, -0.86443363, -0.54223253],
       [-0.90494734,  0.38640518, -0.42410265,  0.24636356],
       [-0.90494734,  0.38640518, -0.42410265,  0.24636356]])

In [163]:
MLP.forward_prop()

In [149]:
a=np.matrix([[1],[2],[3],[4]])

In [152]:
np.array(a)*(1-np.array(a))

array([[  0],
       [ -2],
       [ -6],
       [-12]])

In [156]:
np.array(a)*np.array(a)

array([[ 1],
       [ 4],
       [ 9],
       [16]])

In [147]:
1-np.array(MLP.layers[-1].inputs)

array([[ 0.49491751],
       [ 0.49504872],
       [ 0.49825912],
       [ 0.49816858],
       [ 0.49816858]])

In [165]:
MLP.layers[-2].error

matrix([[ 0.01374095, -0.04431635,  0.07261224, -0.00525996],
        [-0.01720501,  0.05548838, -0.0909176 ,  0.00658599],
        [-0.01720736,  0.05549596, -0.09093002,  0.00658689],
        [ 0.01372371, -0.04426074,  0.07252113, -0.00525336],
        [-0.01720866,  0.05550017, -0.09093692,  0.00658739]])

In [164]:
MLP.backprop()

ValueError: shapes (5,4) and (5,4) not aligned: 4 (dim 1) != 5 (dim 0)

In [118]:
[x.back_delta.shape for x in MLP.layers[1:]]

[(5, 4), (5, 4), (5, 4), (5, 1)]

In [116]:
MLP.layers[3].back_delta.shape

(5, 4)

In [74]:
MLP.layers[-1].backprop(0.1)

In [89]:
MLP.layers[-2].backprop(MLP.layers[-1].back_delta,0.1)

In [92]:
MLP.layers[-2].back_delta.sum(axis=0)

array([-0.02054972, -0.01971989, -0.02422923,  0.01524808])

In [86]:
MLP.layers[-2].inputs.T.dot(MLP.layers[-1].back_delta)

array([[ 0.08092776],
       [ 0.17808934],
       [ 0.19051578],
       [ 0.17255441]])

In [81]:
(MLP.layers[-1].back_delta.dot(MLP.layers[-2].bias.T+MLP.layers[-2].weights.T))

array([[ 0.03241918,  0.03029239,  0.04073021, -0.02268814],
       [-0.0537301 , -0.05020525, -0.06750444,  0.03760231],
       [-0.05369204, -0.05016968, -0.06745661,  0.03757567],
       [ 0.03221911,  0.03010544,  0.04047885, -0.02254812],
       [-0.05373424, -0.05020911, -0.06750964,  0.03760521]])

In [54]:
for i in xrange(60000):
    MLP.forward_prop()
    MLP.backprop()
    if (i % 10000) ==0:
        print(MLP.layers[-1].error)

[[-0.24423168]
 [ 0.75524007]
 [ 0.75597924]
 [-0.24441657]
 [ 0.75558343]]
[[-0.596422  ]
 [ 0.3991783 ]
 [ 0.40143078]
 [-0.60266443]
 [ 0.39733557]]
[[-0.0409066 ]
 [ 0.06697567]
 [ 0.06973347]
 [-0.51148095]
 [ 0.48851905]]
[[-0.01557308]
 [ 0.01684767]
 [ 0.01699568]
 [-0.50048469]
 [ 0.49951531]]
[[-0.01135954]
 [ 0.01176884]
 [ 0.01184875]
 [-0.50021406]
 [ 0.49978594]]
[[-0.00935732]
 [ 0.00951124]
 [ 0.009567  ]
 [-0.50013194]
 [ 0.49986806]]


In [59]:
MLP.layers[-4].error.shape

(5, 4)

In [286]:
MLP.layers[-1].inputs

array([[ 0.00180592],
       [ 0.99776543],
       [ 0.99816564],
       [ 0.00237955]])